## Анализ набора данных с использованием `pandas` (2)

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('Employee.csv')

In [3]:
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [4]:
df.groupby('EverBenched')['LeaveOrNot'].mean()*100

EverBenched
No     33.125749
Yes    45.397490
Name: LeaveOrNot, dtype: float64

Постройте сводную таблицу, показывающую средний возраст сотрудников в каждом городе.

In [5]:
age_by_city = df.pivot_table(values='Age', index='City', aggfunc='mean')
print(age_by_city)

                 Age
City                
Bangalore  29.594255
New Delhi  29.121003
Pune       29.288644


In [6]:
df.pivot_table(values=['Age'], index=['City'])

,Age
City,
Bangalore,29.594255
New Delhi,29.121003
Pune,29.288644


In [7]:
df.groupby('City')['Age'].mean()

City
Bangalore    29.594255
New Delhi    29.121003
Pune         29.288644
Name: Age, dtype: float64

Рассчитайте количество сотрудников по каждому уровню образования и их средний возраст.

In [8]:
df.groupby('Education')['Age'].agg('mean')

Education
Bachelors    29.422938
Masters      29.290951
PHD          29.296089
Name: Age, dtype: float64

In [9]:
df3 = df.groupby('Education')['City'].count()
print('Количество сотрудников: \n', df3.head())
print()
df4 = df.groupby('Education')['Age'].mean()
print('Средний возраст:  \n', df4.head())

Количество сотрудников: 
 Education
Bachelors    3601
Masters       873
PHD           179
Name: City, dtype: int64

Средний возраст:  
 Education
Bachelors    29.422938
Masters      29.290951
PHD          29.296089
Name: Age, dtype: float64


In [10]:
grouped_df = df.groupby('Education')['Age']
result = grouped_df.agg(['size', 'mean']).reset_index()
result.columns = ['Education', 'NumberOfEmployees', 'AverageAge']
print(result)

   Education  NumberOfEmployees  AverageAge
0  Bachelors               3601   29.422938
1    Masters                873   29.290951
2        PHD                179   29.296089


In [11]:
import numpy.ma as ma

df.pivot_table(values=['Age'], index=['Education'], aggfunc=[ma.count, np.mean])

/var/folders/24/z61d72zx27z5ml8twybhwmm40000gn/T/ipykernel_63914/27923053.py:3: FutureWarning: The provided callable <function mean at 0x10d77fec0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(values=['Age'], index=['Education'], aggfunc=[ma.count, np.mean])


,count,mean
,Age,Age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


In [12]:
df.groupby('Education').agg({
    'Education': 'count',
    'Age': 'mean'})

,Education,Age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


In [14]:
df.groupby('Education').agg(num_emp=('Education', 'count'), avg_age=('Age', 'mean'))

,num_emp,avg_age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


Создайте сводную таблицу, показывающую количество мужчин и женщин в каждом уровне оплаты (Payment Tier).

In [15]:
df.pivot_table(values=['Age'], index=['PaymentTier', 'Gender'], aggfunc=[ma.count])

count
                     Age
PaymentTier Gender      
1           Female   131
            Male     112
2           Female   605
            Male     313
3           Female  1139
            Male    2353

In [16]:
pivot_table = pd.crosstab(index=df['PaymentTier'], columns=df['Gender'])
print(pivot_table)

Gender       Female  Male
PaymentTier              
1               131   112
2               605   313
3              1139  2353


In [17]:
df.pivot_table(values = 'PaymentTier', index = 'Gender', aggfunc = 'count')

,PaymentTier
Gender,
Female,1875
Male,2778


In [19]:
pivot=pd.pivot_table(df,index='PaymentTier',columns='Gender',values='Age',aggfunc='count',fill_value=0)
pivot

Gender,Female,Male
PaymentTier,,
1,131,112
2,605,313
3,1139,2353


Составьте сводную таблицу, отражающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от того, были ли они когда-либо исключены из проектов на месяц или более.

In [20]:
df.groupby('EverBenched')['LeaveOrNot'].value_counts(normalize=True)

EverBenched  LeaveOrNot
No           0             0.668743
             1             0.331257
Yes          0             0.546025
             1             0.453975
Name: proportion, dtype: float64

In [22]:
result = df.groupby('EverBenched')['LeaveOrNot'].value_counts(normalize=True).unstack()*100
result.columns = ['Оставшиеся (%)','Покинувшие (%)']
result = result[['Покинувшие (%)']]
result.index.name = None
result.columns.name = None
result

,Покинувшие (%)
No,33.125749
Yes,45.397490


Постройте сводную таблицу, отображающую медианный опыт работы в текущей области в зависимости от уровня образования и пола сотрудников.

In [23]:
pivot=pd.pivot_table(df,index='Education',columns='Gender',values='ExperienceInCurrentDomain',aggfunc='median')
pivot

Gender,Female,Male
Education,,
Bachelors,3.0,3.0
Masters,2.0,3.0
PHD,3.0,3.0


In [24]:
df.pivot_table(
    values='ExperienceInCurrentDomain',  
    index=['Education', 'Gender'],       
    aggfunc='median'                     
)

ExperienceInCurrentDomain
Education Gender                           
Bachelors Female                        3.0
          Male                          3.0
Masters   Female                        2.0
          Male                          3.0
PHD       Female                        3.0
          Male                          3.0

In [25]:
df.pivot_table(values=['ExperienceInCurrentDomain'], index=['Education', 'Gender'], aggfunc=[np.median])

/var/folders/24/z61d72zx27z5ml8twybhwmm40000gn/T/ipykernel_63914/429795385.py:1: FutureWarning: The provided callable <function median at 0x10d8a6660> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df.pivot_table(values=['ExperienceInCurrentDomain'], index=['Education', 'Gender'], aggfunc=[np.median])


median
                 ExperienceInCurrentDomain
Education Gender                          
Bachelors Female                       3.0
          Male                         3.0
Masters   Female                       2.0
          Male                         3.0
PHD       Female                       3.0
          Male                         3.0

Рассчитайте количество сотрудников по каждому году присоединения к компании и их средний возраст.

In [26]:
empl=df.groupby('JoiningYear').agg(count_empl=('Age','count'),mean_empl=('Age','mean'))
empl

,count_empl,mean_empl
JoiningYear,,
2012,504,29.603175
2013,669,29.008969
2014,699,29.290415
2015,781,29.476312
2016,525,29.552381
2017,1108,29.416968
2018,367,29.525886


In [27]:
result = df.groupby('JoiningYear').agg(
    Count=('Age', 'count'),  
    AverageAge=('Age', 'mean')  
).reset_index()
result

,JoiningYear,Count,AverageAge
0,2012,504,29.603175
1,2013,669,29.008969
2,2014,699,29.290415
3,2015,781,29.476312
4,2016,525,29.552381
5,2017,1108,29.416968
6,2018,367,29.525886


Создайте сводную таблицу, показывающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от уровня образования и города, в котором они работают.

In [28]:
df.pivot_table(values = "Age", index = "JoiningYear", aggfunc = ['count', 'mean'])

,count,mean
,Age,Age
JoiningYear,,
2012,504,29.603175
2013,669,29.008969
2014,699,29.290415
2015,781,29.476312
2016,525,29.552381
2017,1108,29.416968
2018,367,29.525886


In [29]:
import numpy.ma as ma

df.pivot_table(values=['Age'], index=['JoiningYear'], aggfunc=[ma.count, np.mean])

/var/folders/24/z61d72zx27z5ml8twybhwmm40000gn/T/ipykernel_63914/3259772935.py:3: FutureWarning: The provided callable <function mean at 0x10d77fec0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(values=['Age'], index=['JoiningYear'], aggfunc=[ma.count, np.mean])


,count,mean
,Age,Age
JoiningYear,,
2012,504,29.603175
2013,669,29.008969
2014,699,29.290415
2015,781,29.476312
2016,525,29.552381
2017,1108,29.416968
2018,367,29.525886


Составьте сводную таблицу, отображающую средний возраст сотрудников, которые были когда-либо исключены из проектов, и тех, кто не был, в зависимости от уровня оплаты.

In [35]:
pivot_table = df.pivot_table(
    values='Age', 
    index='PaymentTier', 
    columns='EverBenched',  
    aggfunc='mean' 
).reset_index()
pivot_table.columns.name = None  
pivot_table = pivot_table.rename(columns={
    'No': 'Средний возраст (не исключались из проектов)',
    'Yes': 'Средний возраст (исключались из проектов)'
})
pivot_table

,PaymentTier,Средний возраст (не исключались из проектов),Средний возраст (исключались из проектов)
0,1,29.644444,28.944444
1,2,29.204848,28.989247
2,3,29.460160,29.217984


Постройте сводную таблицу, показывающую средний возраст сотрудников, которые покинули компанию в следующие два года, в зависимости от города и опыта работы в текущей области.

In [33]:
pivot=pd.pivot_table(df,index='Education',columns='City',values='LeaveOrNot',aggfunc='mean').multiply(100).round(1)
pivot

City,Bangalore,New Delhi,Pune
Education,,,
Bachelors,24.0,18.1,53.3
Masters,70.2,47.0,41.4
PHD,28.8,25.2,16.7


Создайте сводную таблицу, отображающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от уровня оплаты и того, были ли они когда-либо исключены из проектов на месяц или более.